<H1> Storage for Shared Perspectives </H1>

This Notebook contains the code executed to create the database, but it requires that Microsoft SQL Server is configured first (the database, permissions, installing the drivers required, etc.) 

For other RDBMS, there may be small changes in notation.

I used MSSQL because it allows for *User-Defined Table Types* (see https://technet.microsoft.com/en-us/library/bb522526%28v=sql.105%29.aspx?f=255&MSPPError=-2147217396). At the beginning of my research I thought that I could use this to create the *Perceptual Tuple Relations*, enforcing certain conditions and characteristics. However, a key point of these tables is that they depend on the Extraction Method used to obtain the tuple from the source document, (like we are using doc2vec from Amazon reviews in this particular case), and every method will result in tuples with different Perceptual Features and value types.

In [1]:
import pyodbc
import gzip
from collections import defaultdict
import gensim.models.doc2vec
import numpy as np
import time
from collections import defaultdict, Counter
import pickle
from amazon.api import AmazonAPI 
import pyclust
import tmdbsimple as tmdb

<H2>CONNECT TO MSSQL</H2>

In [2]:
config = dict(server=   'DESKTOP-C38DL4M', 
              port=      1433,                  
              database= 'thesis',
              username= 'manuel',
              password= 'password')

connection_string = ('SERVER={server},{port};'   +
                     'DATABASE={database};'      +
                     'UID={username};'           +
                     'PWD={password}')

This is the setup for the connection, and the driver that we used: ODBC Driver 13 for SQL Server

In [4]:
conn = pyodbc.connect(
    r'DRIVER={ODBC Driver 13 for SQL Server};' +
    connection_string.format(**config)
    )

cursor = conn.cursor()

<H3>CREATING THE TABLES</H3>

In [22]:
# Create movie table
cursor.execute('''CREATE TABLE dbo.movies  (id    varchar(50)  NOT NULL PRIMARY KEY, 
                                            title varchar(200),
                                            year  int)''')

Later on, when we had to create the actual prototype we noticed that the url for the poster was required, along with the movie overview. This could also be done properly in the creation of the table, in the cell above.

In [187]:
cursor.execute('''ALTER TABLE movies ADD url varchar(100) ''')
cursor.execute('''ALTER TABLE movies ADD overview varchar(MAX) ''')

Here we create the tables for the vectors extracted with doc2vec. As we can see, for the doc2vec the Perceptual Attributes are just called d0, d1, ..., d99, since they have no meaning for us.

In [23]:
# Create review doc2vec vector table and add all dimensions
cursor.execute('''CREATE TABLE dbo.movie2vec (id  varchar(50)  NOT NULL PRIMARY KEY, 
                                              pid  varchar(50) NOT NULL FOREIGN KEY REFERENCES movies(id)) ''')
for x in range(0, 100):
    cursor.execute('''ALTER TABLE movie2vec ADD d%s float;''' %x)
    
cursor.execute('''ALTER TABLE movie2vec ADD cluster int ''')

Now we create the tables for the Shared Perspective Tuples. Eventhough we have not calculated them yet, we know that they will be an existing Perceptual Tuple, so we can simply refer to it with the rid (review_id, constructed with the reviewer id and the item id), create a cid (cluster_id) and the number of current members in its Shared Perspective.

One is for multiple Shared Perspectives, the other one we force it into a single Perspective...that is what most sistems that deal with ratings or reviews do. You can read more about Shared Perspectives, and the Evaluation section for this.

In [25]:
# Create shared perspective doc2vec vector table and add all dimensions
cursor.execute('''CREATE TABLE dbo.sharedvec (rid  varchar(50)  NOT NULL FOREIGN KEY REFERENCES movie2vec(id),
                                              cid int,
                                              members int)''')

In [5]:
# Create shared perspective doc2vec vector table and add all dimensions
cursor.execute('''CREATE TABLE dbo.sharedvec1 (rid  varchar(50)  NOT NULL FOREIGN KEY REFERENCES featurevec(id),
                                              cid int,
                                              members int)''')

Finally we have to commit the changes made and close the connection.

In [11]:
conn.commit()
conn.close()

<H2>POPULATING THE DATABASE</H2>

For this next section, we may need some steps to be executed first, such as obtaining the Perceptual Tuples, grouping the Shared Perspectives, etc. We might have to connect again to the database.

The first thing we have to introduce is the ASIN or Amazon ID of the movies in our dataset.

In [12]:
full_reviews = []
with gzip.open('data/reviews_Movies_and_TV_5.json.gz') as f:
    for line in f:
        full_reviews.append(json.loads(line))
print('Total reviews in dataset: ', len(full_reviews))

items = defaultdict(int)
 
for review in full_reviews:
    asin = review['asin']
    items[asin] += 1

print('Total movies and TV items in dataset: ', len(items))

Total reviews in dataset: 1697533
Total movies and TV items in dataset: 50052


In [ ]:
for key in items:
    cursor.execute('INSERT INTO movies VALUES (?)', (str(key)))

<H3> Obtain Movie Names from Amazon </H3>

To access the Amazon API, we require certain authorization first (for more information, check https://docs.aws.amazon.com/AWSECommerceService/latest/DG/Welcome.html).
Then we can connect to the API, and ask for some information using the ASIN or identifier.

In [181]:
amazon = AmazonAPI(AMAZON_ACCESS_KEY, AMAZON_SECRET_KEY, AMAZON_ASSOC_TAG)

In [229]:
product = amazon.lookup(ItemId='B0001CNRRU')
product.title

'21 Grams'

With the name and id we can update the database and add the name for every item.

In [ ]:
movie = '21 Grams'
movie_id = 'B0001CNRRU'
cursor.execute('''UPDATE movies set title = ? WHERE id= ? ''', (str(movie), str(movie_id)))

We can automate the process for all the movies in the database, with a sleep of half second, so the API won't kick us out.

In [ ]:
cursor.execute('SELECT * FROM movies')  
table = cursor.fetchall()
for row in table:
    if not row[1]:
        movie_id = row[0]
        try:
            product = amazon.lookup(ItemId=movie_id)
            movie = product.title
        except Exception:
            movie = 'None'
            pass       
        cursor.execute('''UPDATE movies set title = ? WHERE id= ? ''', (str(movie), str(movie_id)))
        time.sleep(0.5) 

<H3> Only Movies, the definitive list </H3>

Amazon does not separate series, collections, concerts or other videos from movies, so we do it by hand. We create a list of movies that we use for the rest of the processing and applications, conviniently called: just_movies

In [5]:
# Definitive list of only movies that have poster

just_movies = []
cursor.execute("SELECT * FROM movies")
movies = cursor.fetchall()
for movie in movies:
    if not 'season' in str(movie[1]).lower():
        if not 'series' in str(movie[1]).lower():
            if not 'collect' in str(movie[1]).lower():
                if not 'pack' in str(movie[1]).lower():
                    if movie[3] != None:
                        just_movies.append(movie[0])
        
len(just_movies)

2042

<H3> Obtain Movie Release Year, Overview and Poster from The Movie Database (TMDb) </H3>

Once we have the movie titles from Amazon, we can get even more useful information from The Movie Database. For this, we have to get an API key and follow some simple instructions. For more info visit https://pypi.python.org/pypi/tmdbsimple/2.0.0

In [6]:
tmdb.API_KEY = 'key-code'

In [10]:
search = tmdb.Search()
response = search.movie(query='The Green Mile')
for s in search.results:
    print(s['title'], s['id'], s['release_date'], s['popularity'], movie['poster_path'])

The Green Mile 497 1999-12-10 39.169279 /5JU9ytZJyR3zmClGmVm9q4Geqbd.jpg


Then we can use this search from TMDb to obtain the release year, poster URL and the Overview of our movies. Unfortunately, some movies have product names (like "The Green Mile [VHS]" or "Hercules (remastered)" that TMDb cannot process, so we have to cut the title wherever those characters appear.

In [ ]:
cursor.execute('SELECT * FROM movies')  
table = cursor.fetchall()

stops = ['(', '[',  '-', "Director's Cut" , '/', '!', '-', ':', '(+', '(']
base_url = 'http://image.tmdb.org/t/p/'
size = 'w185'

for row in table:
    if row[0] in just_movies:
        if row[1] == None:
            title = 'Not Found'
        else:
            title = row[1]
            cut_here = [e for e in stops if e in title]
            if cut_here:
                s = title
                title = s[:s.index(cut_here[0])]
            search = tmdb.Search()
            response = search.movie(query = title)
            try:
                movie = search.results[0]
                url = base_url + size +'/' + movie['poster_path']
                print(len(movie['overview']))
                cursor.execute('''UPDATE movies set title = ?, year = ?, url = ?, overview = ? WHERE id = ? ''', (title, str(movie['release_date'][0:4]), str(url), str(movie['overview']), str(row[0])))
            except Exception:
                pass

We can even download the movie posters using the url and save them into a movie_posters folder

In [ ]:
# Get poster URL for the movies

import urllib.request

cursor.execute('SELECT * FROM movies')  
table = cursor.fetchall()

for movie in table:
    if movie[0] in just_movies:
        if movie[3] != None:
            urllib.request.urlretrieve(movie[3], "movie_posters/%s.jpg" %(movie[0]))

<H3>Input Perceptual Tuples from Doc2vec Model </H3>

For this section, we need to have the vectors for every review. We can load the trained model, and then get the vector for every review.

In [ ]:
model = Doc2Vec.load("data/movies_dbow10epoch.doc2vec")

In [ ]:
vectors100d = []
dimensions = 100

for review in full_reviews:         
    vector = []
    review_id = (review['reviewerID']+'|'+review['asin'])
    vector.append(review_id)
    vector.append(review['asin'])
    for y in range(dimensions):
        a = str(model.docvecs[review_id][y])    
        vector.append(a)
    vectors100d.append(vector)

In [47]:
for x in range(len(vectors100d)):
    v = vectors100d[x]
    v = ','.join(v)  
    v = v + ', NULL'
    cursor.execute('''INSERT INTO dbo.movie2vec VALUES  (%s); '''%v)
    if x % 50000 == 1:
        print(x,'reviews introduced') #just to see progress

1 reviews introduced
50001 reviews introduced
100001 reviews introduced
150001 reviews introduced
200001 reviews introduced
250001 reviews introduced
300001 reviews introduced
350001 reviews introduced


<H2> Shared Perspectives in Database </H2>

Calculate and insert the Shared Perspective Tuples using one perspective per movie, into sharedvec1 table.

In [10]:
x = 0
start_time = time.time()
cursor.execute("SELECT id FROM movies")
movies = cursor.fetchall()
for movie in movies:
    print(movie)
    movie = movie[0]
    V = []
    cursor.execute("SELECT * FROM movie2vec WHERE pid = ?", (movie))
    for entry in cursor:
        v = entry[2:102]        
        V.append(v)
    V = np.array(V)
    kmd = pyclust.KMedoids(distance='cosine', n_clusters=1,  n_trials=20)
    if len(V) > 3:
        kmd.fit(V)        
        clusters = defaultdict(int)
        for label in kmd.labels_:
            clusters[label] += 1
        i = 0
        cursor.execute("SELECT * FROM movie2vec WHERE pid = ?", (movie))
        table = cursor.fetchall()
        for row in table:
            cursor.execute("UPDATE movie2vec SET cluster = %s  WHERE id = '%s'" %(kmd.labels_[i], str(row[0])))
            i += 1            
        i = 0
        for x in kmd.centers_:
            cursor.execute("SELECT id FROM movie2vec WHERE d0 = ? AND d80 = ?", (x[0], x[80]))
            id = cursor.fetchone()
            cursor.execute("SELECT cluster1 FROM movie2vec WHERE d0 = ? AND d80 = ?", (x[0], x[80]))
            cid = cursor.fetchone()
            cursor.execute('''INSERT INTO sharedvec1 VALUES (?, ?, ?); ''', (id[0], cid[0], clusters[i]))
            i += 1
    else:
        print(movie)

('0307514161', )
('0767020308', )
('0767726227', )
('0767736680', )
('0767802470', )
('0767802497', )
('0767802519', )
('0767802551', )
('0767802594', )
('0767802624', )
('0767802659', )
('0767802683', )
('0767802802', )
('0767803434', )
('0767805267', )
('0767805712', )
('0767808673', )
('076780922X', )
('0767809246', )
('0767809254', )
('0767809262', )
('0767809270', )
('0767809688', )
('0767810864', )
('0767811100', )
('0767812166', )
('0767812417', )
('0767812778', )
('0767813871', )
('0767814037', )
('076781505X', )
('0767816234', )
('0767817478', )
('0767817656', )
('0767817664', )
('0767817710', )
('0767817753', )
('0767819438', )
('0767819586', )
('0767821556', )
('0767821785', )
('0767823729', )
('0767824407', )
('0767825411', )
('0767827716', )
('0767830520', )
('0767834739', )
('0767836286', )
('0767836316', )
('0767836324', )
('0767836359', )
('0767839129', )
('0767851013', )
('0767853636', )
('0769403468', )
('0780018664', )
('0780020685', )
('0780020715', )
('0780020723',

('6300214710', )
('6300215512', )
('6300215520', )
('6300215555', )
('6300215598', )
('6300215628', )
('6300215644', )
('6300215717', )
('6300215733', )
('6300215954', )
('6300216012', )
('6300216047', )
('6300216233', )
('6300216268', )
('630021639X', )
('6300216403', )
('6300216500', )
('6300216543', )
('6300216632', )
('6300216721', )
('6300216748', )
('6300216845', )
('6300216977', )
('6300218171', )
('6300246787', )
('6300246981', )
('6300247031', )
('6300247090', )
('6300247104', )
('6300247236', )
('6300248135', )
('6300248550', )
('6300248569', )
('6300249964', )
('6300251004', )
('6300251217', )
('6300252051', )
('6300262731', )
('6300263681', )
('6300267830', )
('6300267962', )
('6300267997', )
('6300268179', )
('6300268470', )
('6300268748', )
('6300268853', )
('6300269043', )
('630026923X', )
('6300269329', )
('6300269477', )
('6300270025', )
('6300270033', )
('6300270122', )
('6300270270', )
('630027120X', )
('6300271285', )
('6300271617', )
('630027179X', )
('6300274195',

('B00000JMQC', )
('B00000JQU8', )
('B00000JQUB', )
('B00000JRUB', )
('B00001QEE2', )
('B00001U0BK', )
('B00001U0DX', )
('B00001ZWTS', )
('B0000203YR', )
('B000021Y6M', )
('B000021Y74', )
('B000028U3R', )
('B00002E233', )
('B000031WD7', )
('B000035P81', )
('B000035Z2S', )
('B0000399WC', )
('B00003BDZW', )
('B00003CWM2', )
('B00003CWN2', )
('B00003CWPL', )
('B00003CWRX', )
('B00003CX43', )
('B00003CX73', )
('B00003CX9I', )
('B00003CXBK', )
('B00003CXDI', )
('B00003CXDR', )
('B00003CXE4', )
('B00003CXFU', )
('B00003CXH0', )
('B00003CXHJ', )
('B00003CXHR', )
('B00003CXI6', )
('B00003CXIF', )
('B00003CXIU', )
('B00003CXJ1', )
('B00003CXJ3', )
('B00003CXJC', )
('B00003CXK9', )
('B00003CXKJ', )
('B00003CXKM', )
('B00003CXKR', )
('B00003CXLV', )
('B00003CXMO', )
('B00003CXN3', )
('B00003CXO0', )
('B00003CXP7', )
('B00003CXPC', )
('B00003CXPJ', )
('B00003CXPP', )
('B00003CXPS', )
('B00003CXPV', )
('B00003CXQM', )
('B00003CXQR', )
('B00003CXQX', )
('B00003CXRA', )
('B00003CXS4', )
('B00003CXS7',

('B0006D3HGA', )
('B0006FO9B0', )
('B0006GAI6O', )
('B0006GAO18', )
('B0006GAO54', )
('B0006IIKQW', )
('B0006IO778', )
('B0006IUD9Y', )
('B0006IUDXA', )
('B0006IUE16', )
('B0006JMLQQ', )
('B0006TPE4C', )
('B0007A0F4E', )
('B0007CNXUK', )
('B0007IO6PA', )
('B0007KIFI2', )
('B0007N1BBC', )
('B0007N1JC8', )
('B0007OCG4W', )
('B0007OCG56', )
('B0007P0X9G', )
('B0007P0XBO', )
('B0007P0Y7C', )
('B0007PALGG', )
('B0007PICAI', )
('B0007Q6VXC', )
('B0007R4T3U', )
('B0007TKH66', )
('B0007UQ264', )
('B0007V6IT4', )
('B0007XBM5W', )
('B0007Y08QA', )
('B0007Y3XRG', )
('B0007Z0NY6', )
('B0007Z2556', )
('B0008FXT1Y', )
('B0009231SC', )
('B00092ZLS0', )
('B00092ZMDY', )
('B00094ARN2', )
('B00094AS9A', )
('B00094AS9U', )
('B000957O7S', )
('B0009A5MUO', )
('B0009B16TE', )
('B0009ETCUQ', )
('B0009ETCVA', )
('B0009FU0ZQ', )
('B0009I7NGW', )
('B0009NZ2VU', )
('B0009OL7Z4', )
('B0009RCPUW', )
('B0009S2T0M', )
('B0009S4IHY', )
('B0009UC7H0', )
('B0009ULBGS', )
('B0009VBTQY', )
('B0009WPM1Q', )
('B0009X763C',

('B0021L8UXA', )
('B0021L8V34', )
('B0021L8V3Y', )
('B0024396EW', )
('B0024NSFYY', )
('B00266E6G0', )
('B00275EHBY', )
('B0027FG29Y', )
('B0027P94CQ', )
('B0027VST2Q', )
('B00280LZAE', )
('B00288KNJU', )
('B002935GMS', )
('B0029W2V9U', )
('B002AL2TYC', )
('B002AT4K9G', )
('B002BFBAUG', )
('B002BVYBJW', )
('B002BWP2IK', )
('B002C6VMKC', )
('B002DGTAGY', )
('B002DKGWSE', )
('B002DU39GW', )
('B002E2M5IC', )
('B002GP7ZWI', )
('B002HRF68A', )
('B002I9Z8GM', )
('B002IKIHE6', )
('B002IRYYCI', )
('B002JCSWV6', )
('B002JIOOCQ', )
('B002JT69IM', )
('B002JVWQSW', )
('B002JVWR9U', )
('B002JVWRAO', )
('B002KCO6QA', )
('B002KISB56', )
('B002KSA4F6', )
('B002LMSWN2', )
('B002LMV7Q6', )
('B002N5N4M6', )
('B002P413IC', )
('B002P7UCJ0', )
('B002QEHPQA', )
('B002QW7AJY', )
('B002R0EIVI', )
('B002R0ELCY', )
('B002SG7Z7A', )
('B002TVQ48A', )
('B002TVQ4GW', )
('B002UJIY70', )
('B002UNMW7O', )
('B002UOMGZQ', )
('B002USF1VS', )
('B002UZCJ8Y', )
('B002V3HT2G', )
('B002VECLVO', )
('B002VECM1S', )
('B002VECM3Q',

('B00C7BZYRO', )
('B00C7C00JU', )
('B00CC3464E', )
('B00CENSU1W', )
('B00CJ5BG8Y', )
('B00CS5RBPM', )
('B00CTMQ7WM', )
('B00CX7TOOG', )
('B00CYQXGMW', )
('B00CZB9BCU', )
('B00D2CYU2W', )
('B00D6MB83W', )
('B00DB43PLI', )
('B00DCLT8RA', )
('B00DL46ZN8', )
('B00DL477I0', )
('B00DL47ORO', )
('B00DL47YY2', )
('B00DS7F6RY', )
('B00DTPRZ0G', )
('B00DW5IKN4', )
('B00DXP7L88', )
('B00E00OGE2', )
('B00E1LQEFE', )
('B00E5G03I4', )
('B00ECR7KX2', )
('B00EO10IEE', )
('B00F37VHPM', )
('B00FPPQYXM', )
('B00FQY4C86', )
('B00FRILRL6', )
('B00G2P79BU', )
('B00G7QPXAI', )
('B00GD9GNEY', )
('B00GEKO51U', )
('B00GMV8KGW', )
('B00H5RYIBI', )
('B00H7KJRVY', )
('B00H7KJTCG', )
('B00H9HZGQ0', )
('B00H9LHVHI', )
('B00HEPC0TS', )
('B00HHYF570', )
('B00HLSW6TW', )
('B00HNGZHDE', )
('B00HUAH14Q', )
('B00HUCF420', )
('B00HUCF7AO', )
('B00HX0KIAM', )
('B00JA3RPAG', )
('B00JAQJMJ0', )


Calculate and insert the Shared Perspective Tuples using 3 Shared Perspectives per movie, into sharedvec table.

In [ ]:
x = 0
start_time = time.time()
cursor.execute("SELECT id FROM movies")
movies = cursor.fetchall()
for movie in movies:
    print(movie)
    movie = movie[0]
    V = []
    cursor.execute("SELECT * FROM movie2vec WHERE pid = ?", (movie))
    for entry in cursor:
        v = entry[2:102]        
        V.append(v)
    V = np.array(V)
    kmd = pyclust.KMedoids(distance='cosine', n_clusters=3,  n_trials=20)
    if len(V) > 3:
        kmd.fit(V)        
        clusters = defaultdict(int)
        for label in kmd.labels_:
            clusters[label] += 1
        i = 0
        cursor.execute("SELECT * FROM movie2vec WHERE pid = ?", (movie))
        table = cursor.fetchall()
        for row in table:
            cursor.execute("UPDATE movie2vec SET cluster = %s  WHERE id = '%s'" %(kmd.labels_[i], str(row[0])))
            i += 1            
        i = 0
        for x in kmd.centers_:
            cursor.execute("SELECT id FROM movie2vec WHERE d0 = ? AND d80 = ?", (x[0], x[80]))
            id = cursor.fetchone()
            cursor.execute("SELECT cluster FROM movie2vec WHERE d0 = ? AND d80 = ?", (x[0], x[80]))
            cid = cursor.fetchone()
            cursor.execute('''INSERT INTO sharedvec VALUES (?, ?, ?); ''', (id[0], cid[0], clusters[i]))
            i += 1
    else:
        print(movie)

Save some data into pickle files for easier future use.

In [6]:
with open(r'data/review_ids.pickle', 'wb') as output_file:
    pickle.dump(review_ids, output_file)  
    
with open(r'data/just_movies.pickle', 'wb') as output_file:
    pickle.dump(just_movies, output_file)  

overviews = {}
cursor.execute('SELECT * FROM movies')  
table = cursor.fetchall()
for row in table:
    if row[0] in just_movies:
        overviews[row[0]] = row[4]
        
with open(r'data/overviews.pickle', 'wb') as output_file:
    pickle.dump(overviews, output_file)   
    
poster_urls = {}
cursor.execute('SELECT * FROM movies')  
table = cursor.fetchall()
for row in table:
    if row[0] in just_movies:
        poster_urls[row[0]] = row[3]
        
with open(r'data/poster_urls.pickle', 'wb') as output_file:
    pickle.dump(poster_urls, output_file)  

<H3> Shared Perspective Tuple Catalog </H3>

We can create this catalog to 

In [11]:
cursor.execute('''SELECT movie2vec.*, sharedvec.members FROM sharedvec INNER JOIN movie2vec
                    ON sharedvec.rid = movie2vec.id 
                    WHERE movie2vec.pid = ? ORDER BY movie2vec.pid, sharedvec.members DESC ''', str('0780628799'))
perspectives = cursor.fetchall()

In [35]:
cursor.execute('''SELECT * FROM movie2vec WHERE pid = ? ''', str('0780628799'))
perspectives = cursor.fetchall()

In [27]:
cursor.execute('''SELECT * FROM movie2vec WHERE pid = ? and cluster = 1''', str('0780628799'))
perspectives = cursor.fetchall()

In [36]:
cursor.execute('''SELECT * FROM movie2vec WHERE id = ? ''', str('A19V0VJMKKRMC8|0780628799'))
base = cursor.fetchone()